# Data Scientist Intern

### Объяснение highway_cost (стоимости доставки):

Когда заказ доставляется из склада, то списывается стоимость доставки. У каждого склада есть определенный тариф, определяющий стоимость доставки. Это тариф имеет размерность стоимость доставки на единицу товара.

Например для склада "гиперборея" стоимость тарифа составляет 20 рублей на единицу товара. Причем тариф не зависит от того какой именно товар мы заказали. Так, если мы заказали 5 зеленых пластинок и 3 билета в Израиль,
то стоимость тарифа будет 20*(5+3) = 160 рублей.

### Уточнение по виду отчета:

В поле "products" не обязательно могут быть только уникальные значения наименовая товаров ("product"). Иногда (как в примере выше) названия товаров могут повторяться в поле "products".

Однако поле "products" не может быть пустым

### Доп пояснение:

доходом с товара является цена продажи * количество товара

расходом является тариф для данного склада * количество товара

прибылью является доход - расход

# Задания:

In [217]:
#импортируем необходимые библиотеки
import pandas as pd

In [218]:
#прочитаем наш json
df = pd.read_json("./trial_task.json", orient="columns")
df.head(5)

,order_id,warehouse_name,highway_cost,products
0,11973,Мордор,-70,"[{'product': 'ломтик июльского неба', 'price':..."
1,62239,хутор близ Диканьки,-15,"[{'product': 'билет в Израиль', 'price': 1000,..."
2,85794,отель Лето,-50,"[{'product': 'зеленая пластинка', 'price': 10,..."
3,33684,Мордор,-30,"[{'product': 'билет в Израиль', 'price': 1000,..."
4,25824,отель Лето,-75,"[{'product': 'автограф Стаса Барецкого', 'pric..."


In [219]:
#развернем столбец projects, который содержит список словарей
df_exploded = df.explode("products", ignore_index = True)
df_exploded.head(5)

,order_id,warehouse_name,highway_cost,products
0,11973,Мордор,-70,"{'product': 'ломтик июльского неба', 'price': ..."
1,11973,Мордор,-70,"{'product': 'билет в Израиль', 'price': 1000, ..."
2,11973,Мордор,-70,"{'product': 'статуэтка Ленина', 'price': 200, ..."
3,62239,хутор близ Диканьки,-15,"{'product': 'билет в Израиль', 'price': 1000, ..."
4,85794,отель Лето,-50,"{'product': 'зеленая пластинка', 'price': 10, ..."


In [220]:
#преобразуем словари в отдельные столбцы для удобства чтения данных
pd.json_normalize(df_exploded['products'])

,product,price,quantity
0,ломтик июльского неба,450,1
1,билет в Израиль,1000,3
2,статуэтка Ленина,200,3
3,билет в Израиль,1000,1
4,зеленая пластинка,10,2
...,...,...,...
192,автограф Стаса Барецкого,600,1
193,ломтик июльского неба,450,1
194,плюмбус,250,2
195,плюмбус,250,1


In [221]:
#сконкатенируем df_exploded и нормализированный датасет
df = pd.concat([df_exploded.drop('products', axis = 1), 
pd.json_normalize(df_exploded['products'])], axis = 1)

df.head(5)

,order_id,warehouse_name,highway_cost,product,price,quantity
0,11973,Мордор,-70,ломтик июльского неба,450,1
1,11973,Мордор,-70,билет в Израиль,1000,3
2,11973,Мордор,-70,статуэтка Ленина,200,3
3,62239,хутор близ Диканьки,-15,билет в Израиль,1000,1
4,85794,отель Лето,-50,зеленая пластинка,10,2


1. Найти тариф стоимости доставки для каждого склада.

In [222]:
df_tarif = (
    df.merge(df.groupby('order_id')['quantity'].sum(), on='order_id')
    .assign(tarif=lambda x: x['highway_cost'] / x['quantity_y'])[['warehouse_name', 'tarif']]
    .drop_duplicates()
    .reset_index(drop=True)
)
df_tarif

,warehouse_name,tarif
0,Мордор,-10.0
1,хутор близ Диканьки,-15.0
2,отель Лето,-25.0
3,остров невезения,-5.0
4,гиперборея,-20.0


2. Найти суммарное количество , суммарный доход , суммарный расход и суммарную прибыль для каждого товара (представить как таблицу со столбцами
'product', 'quantity', 'income', 'expenses', 'profit').

In [223]:
df_good = (
    df.merge(df.groupby('product')['quantity'].sum(), on='product')
    .drop('quantity_x', axis= 1)
    .rename(columns={'quantity_y': 'quantity'})
    .assign(income=lambda x: x['quantity'] * x['price'])
    .merge(df_tarif.groupby('warehouse_name')['tarif'].sum().rename('tarif'), on='warehouse_name')
    .assign(expenses=lambda x: x['tarif'] * x['quantity'])
    .assign(profit=lambda x: x['income'] + x['expenses'])
    [['product', 'quantity', 'income', 'expenses', 'profit']]
    .drop_duplicates()
    .reset_index(drop=True)
)
df_good



,product,quantity,income,expenses,profit
0,ломтик июльского неба,42,18900,-420.0,18480.0
1,билет в Израиль,58,58000,-580.0,57420.0
2,статуэтка Ленина,68,13600,-680.0,12920.0
3,зеленая пластинка,61,610,-610.0,0.0
4,автограф Стаса Барецкого,48,28800,-480.0,28320.0
5,плюмбус,65,16250,-650.0,15600.0
6,подписка на suppi-блог,33,4950,-330.0,4620.0
7,ломтик июльского неба,42,18900,-840.0,18060.0
8,билет в Израиль,58,58000,-1160.0,56840.0
9,статуэтка Ленина,68,13600,-1360.0,12240.0


3. Составить табличку со столбцами 'order_id' (id заказа) и 'order_profit' (прибыль полученная с заказа). А также вывести среднюю прибыль заказов.

In [229]:
df_profit = df
df_profit = (
    df_profit.assign(income = lambda x: x['price'] * x['quantity'])
    .merge(df_profit.groupby('order_id')['income'].sum().rename('order_income'), on='order_id')
    .assign(profit = lambda x: x['order_income']+x['highway_cost'])
    .drop_duplicates(subset='order_id')[['order_id', 'profit']]
    .reset_index(drop=True)
)
    
df_profit

,order_id,profit
0,11973,3980
1,62239,985
2,85794,-30
3,33684,1980
4,25824,975
...,...,...
95,28106,320
96,98100,1590
97,79293,1175
98,2930,720
